# Importat librerias

In [ ]:
import cv2
import matplotlib
import os
import SRCNN
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.gridspec import GridSpec

# python magic function, displays pyplot figures in the notebook
%matplotlib inline

# Indicate the text size of the matplotlib to plot the history results of the model train
fontText = {'family' : 'Times New Roman',
        'weight' : 'normal',
        'size'   : 20}
fontTitle = {'family' : 'Arial',
        'weight' : 'bold',
        'size'   : 36}
matplotlib.rc('font', **fontText)

## Leer Imagen de entrada para realizar el "zoom" a una parte seleccionada
Se seleccionará a partir de una imagen inicial una parte de esta, a la cual se le hara "zoom" y mediante la aplicación de super resolución se mejorará el zoom realizado

In [ ]:
x = 466                                          # Posicion en x del área que será seleccionada para zoom
y = 343                                          # Posicion en y del área que será seleccionada para zoom
w = 32                                           # Ancho del área de zoom
h = 32                                           # Alto del área de zoom

imgPath = 'AppTest/Saltillo.png'
imgOrig = cv2.imread(imgPath)            # Imagen original a la cual realizaremos zoom
imgLR = np.copy(imgOrig[y:y+h, x:x+w, :])            # Imagen en baja resolución (Área de zoom)

# Creamos el módelo SRCNN
srcnn = SRCNN.SR()

# Factor de escalamiento de la imagen
factor = 4

# Dimensiones de reescalamiento de imagen
dim = (factor*imgLR.shape[0],factor*imgLR.shape[1])

# Predicción de la imagen en alta resolución, la estructura de llamada del método "getImage" es:
# getImage(Imagen, Espacio de color 0=RGB 1=YCrCb, Archivo .h5 de modelo de red entrenado) la imagen debe ser BGR
imgSRCNN = srcnn.getImage(cv2.resize(imgLR,dim,cv2.INTER_CUBIC),SRCNN.SR.ColorSpace.YCrCb,'models/model_YCrCbFactor4.h5')
imgOrig = cv2.cvtColor(imgOrig, cv2.COLOR_BGR2RGB)
imgLR = cv2.cvtColor(imgLR, cv2.COLOR_BGR2RGB)
imgSRCNN = cv2.cvtColor(imgSRCNN, cv2.COLOR_BGR2RGB)

# Definición de estructura de la figura de imagenes de resultado
fig = plt.figure(figsize=(15,15),dpi=200.0, constrained_layout=True)
gs = GridSpec(3, 3, figure=fig)
ax1 = fig.add_subplot(gs[0:2, :])
ax2 = fig.add_subplot(gs[2, 0])
ax3 = fig.add_subplot(gs[2, 1])
ax4 = fig.add_subplot(gs[2, 2])

ax1.imshow(imgOrig)
ax1.imshow(cv2.rectangle(imgOrig, (x,y), (x+w,y+h), [0,255,0], 5))
ax1.axis('off')
ax1.autoscale('on')

ax2.set_title('Area de zoom')
ax2.imshow(imgLR)
ax2.axis('off')
ax2.autoscale('on')

ax3.set_title('Zoom con Int. Bicubica')
ax3.imshow(cv2.resize(imgLR,(4*imgLR.shape[0],4*imgLR.shape[1]),cv2.INTER_CUBIC))
ax3.axis('off')
ax3.autoscale('on')

ax4.set_title('Zoom con SRCNN')
ax4.imshow(imgSRCNN)
ax4.axis('off')
ax4.autoscale('on')

plt.savefig('Resultado.png')

plt.show()